In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [2]:
!pip install transformers[sentencepiece] datasets spacy scipy networkx numpy sent2vec pyngrok pandas nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5d40dd2d998e7e7ced121e85a266aa356b13d65b2bc8581fa4fec9f2f6caaec6
  Sto

In [3]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6089a593a761dade8107e660126101e9769ddfe5ed5cc72e9c0a5ae6e7d450ce
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
# import library yang dibutuhkan
import spacy
import networkx as nx
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast

import re
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**PERINGKASAN BAHASA INGGRIS**

In [5]:
# Load the tokenizer and model
tokenizer_bert_en = BertTokenizer.from_pretrained('distilbert-base-uncased')
model_bert_en = BertModel.from_pretrained('distilbert-base-uncased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['encoder.layer.0.output.dense.weight', 'encoder.layer.7.intermediate.dense.weight', 'encoder.layer.1.attention.output.dense.bias', 'encoder.layer.8.output.LayerNorm.weight', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.6.attention.self.value.weight', 'encoder.layer.4.attention.output.dense.weight', 'encoder.layer.11.attention.self.value.bias', 'encoder.layer.11.output.dense.weight', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.7.attention.self.key.bias', 'encoder.layer.9.attention.self.value.weight', 'encoder.layer.11.output.LayerNorm.weight', 'encoder.layer.6.attention.self.key.weight', 'encoder.layer.11.output.dense.bias', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.

In [6]:
def vector_en(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [7]:
# 1. FUNGSI PRE-PROCESSING
def preprocess_en(text):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = [sent.text.strip() for sent in doc.sents]

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):
    doc = nlp(sentences[i])

    # inisial variabel untuk menampung kata yang bersih
    filtered_tokens = []

    # tokenisasi
    for token in doc:
    # menghilangkan stopword dan karakter yang tidak dibutuhkan
      if token.is_stop or token.is_punct:
        continue
      # lemmatisasi
      filtered_tokens.append(token.lemma_)

    # how to convert list to string
    result = " ".join(filtered_tokens)

    filtered_sentences.append(result)
  return filtered_sentences

In [8]:
# 2. FUNGSI PERINGKASAN EKSTRAKTIF
def extractive_sum_en(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_en(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]
  summary = " ".join(summary)

  # hasil peringkasan ekstraktif
  return summary


In [9]:
# 3. FUNGSI PERINGKASAN ABSTRAKTIF

model_name1 = "google/pegasus-cnn_dailymail"
model1 = PegasusForConditionalGeneration.from_pretrained(model_name1)
tokenizer1 = PegasusTokenizer.from_pretrained(model_name1)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def abstractive_sum_en(text):

  inputs = tokenizer1.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

  summary_ids = model1.generate(inputs, max_length=350, min_length=50, length_penalty=2.0, num_beams=6, early_stopping=True)

  summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)

  return summary

In [11]:
# MERINGKAS TEKS BAHASA INGGRIS
def summy_en(text, n):
  document = text
  # load spacy
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary1 = extractive_sum_en(result, sentences, n)

  # melakukan peringkasan abstraktif
  summary = abstractive_sum_en(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary


In [12]:
text = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [13]:
result_en = summy_en(text, 2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
print(result_en)

Python supports object-oriented programming, functional programming, and procedural programming paradigms.
 Python is a versatile tool for different kinds of applications.
With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields.


**PERINGKASAN BAHASA INDONESIA**

In [15]:
# Load the tokenizer and model
tokenizer_bert_id = BertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model_bert_id = BertModel.from_pretrained('distilbert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of BertModel were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['encoder.layer.0.output.dense.weight', 'encoder.layer.7.intermediate.dense.weight', 'encoder.layer.1.attention.output.dense.bias', 'encoder.layer.8.output.LayerNorm.weight', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.6.attention.self.value.weight', 'encoder.layer.4.attention.output.dense.weight', 'encoder.layer.11.attention.self.value.bias', 'encoder.layer.11.output.dense.weight', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.7.attention.self.key.bias', 'encoder.layer.9.attention.self.value.weight', 'encoder.layer.11.output.LayerNorm.weight', 'encoder.layer.6.attention.self.key.weight', 'encoder.layer.11.output.dense.bias', 'encoder.layer.6.attention.self.query.weight', 'encoder.layer.0.attention.output.LayerNorm.bias'

In [16]:
def vector_id(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [17]:
#1. PRE-PROCESSING
def preprocess_id(text):

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = sent_tokenize(text)

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):

    text = sentences[i]

    # Tokenisasi (memecah kalimat menjadi kata)
    words = word_tokenize(text.lower())

    # Membersihkan tanda baca
    words = [word for word in words if word.isalnum()]

    # Menghapus stopwords
    stop_words = set(stopwords.words("indonesian"))
    words = [word for word in words if word not in stop_words]

    # Melakukan lemmatisasi
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Menggabungkan kembali kata yang telah dibersihkan menjadi kalimat
    result = " ".join(lemmatized_words)

    filtered_sentences.append(result)
  return filtered_sentences

In [18]:
# 2. PERINGKASAN EKSTRAKTIF

def extractive_sum_id(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_id(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]

  # hasil peringkasan ekstraktif

  return " ".join(summary)

In [19]:
# 3. PERINGKASAN ABSTRAKTIF

# membersihkan teks
def text_cleaning(input_string):
    lowercase = input_string.lower()
    remove_link = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)', '', lowercase).replace("&amp;","&")
    remove_bullet = "\n".join([T for T in remove_link.split('\n') if '•' not in T and "baca juga:" not in T])
    remove_accented = unicodedata.normalize('NFKD', remove_bullet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    remove_parentheses = re.sub("([\(\|]).*?([\)\|])", "\g<1>\g<2>", remove_accented)
    remove_punc = re.sub(r"[^\w\d.\s]+",' ', remove_parentheses)
    remove_num_dot = re.sub(r"(?<=\d)\.|\.(?=\d)|(?<=#)\.","", remove_punc)
    remove_extra_whitespace =  re.sub(r'^\s*|\s\s*', ' ', remove_num_dot).strip()
    return ".".join([s for s in remove_extra_whitespace.strip().split('.') if len(s.strip())>10]).replace("_","")

In [20]:
model_name2 = "thonyyy/pegasus_indonesian_base-finetune"
model2 = TFPegasusForConditionalGeneration.from_pretrained(model_name2)
tokenizer2 = PegasusTokenizerFast.from_pretrained(model_name2)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at thonyyy/pegasus_indonesian_base-finetune.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
def abstractive_sum_id(text):
  clean_text = text_cleaning(text)
  inputs = tokenizer2(clean_text, return_tensors = 'tf')
  summary_ids = model2.generate(**inputs, max_new_tokens = 256, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer2.batch_decode(summary_ids, skip_special_tokens=True)

  return summary[0]

In [22]:
# MERINGKAS TEKS BAHASA
def summy_id(text, n):

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  sentences = sent_tokenize(text)

  # 1. PRE-PROCESSING
  result = preprocess_id(text)

  # 2. PERINGKASAN EKSTRAKTIF
  summary1 = extractive_sum_id(result, sentences, n)

  # 3. PERINGKASAN ABSTRAKTIF
  summary = abstractive_sum_id(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary

In [23]:
text = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [24]:
result_id = summy_id(text, 2)

In [25]:
print(result_id)

sampah yang menumpuk di tempat pembuangan akhir sampah di kota bandung, jawa barat, dapat menyebabkan pencemaran lingkungan


**PERINGKASAN BAHASA INGGRIS PENGUJIAN**

In [26]:
# 1. TEXTRANK
def f_textrank_en(text, n):
  document = text
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_en(result, sentences, n)

  return summary

In [27]:
# 2. PEGASUS
def f_pegasus_en(text):
  summary = abstractive_sum_en(text)
  return summary

In [28]:
# 3. HYBRID
def f_hybrid_en(text, n):
  summary = summy_en(text, n)
  return summary

In [29]:
text1 = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [30]:
hasil_textrank_en = f_textrank_en(text1, 3)
hasil_pegasus_en = f_pegasus_en(text1)
hasil_hybrid_en = f_hybrid_en(text1, 2)

In [31]:
hasil_textrank_en

'So why not start learning Python today and see where it takes you? Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications.'

In [32]:
hasil_pegasus_en

'Python is a high-level, interpreted programming language.<n>It is widely used for web development, data analysis, artificial intelligence, and many other applications.<n>Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages.'

In [33]:
hasil_hybrid_en

'Python supports object-oriented programming, functional programming, and procedural programming paradigms.\n Python is a versatile tool for different kinds of applications.\nWith its growing popularity and wide range of applications, Python is an essential skill for many industries and fields.'

**PERINGKASAN BAHASA INDONESIA PENGUJIAN**

In [34]:
# 1. TEXTRANK
def f_textrank_id(text, n):
  sentences = sent_tokenize(text)

  result = preprocess_id(text)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_id(result, sentences, n)

  return summary

In [35]:
# 2. PEGASUS
def f_pegasus_id(text):
  summary = abstractive_sum_id(text)
  return summary

In [36]:
# 3. HYBRID
def f_hybrid_id(text, n):
  summary = summy_id(text, n)
  return summary

In [37]:
text2 = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [38]:
hasil_textrank_id = f_textrank_id(text2, 3)
hasil_pegasus_id = f_pegasus_id(text2)
hasil_hybrid_id = f_hybrid_id(text2, 2)

In [39]:
hasil_textrank_id

'Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. \n\nSampah selalu kita temukan mengotori lingkungan di sekitar kita.'

In [40]:
hasil_pegasus_id

'sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. padahal sampah jelas jelas membuat lingkungan tidak nyaman tetapi anehnya membuat lingkungan masih jauh dari cukup banyak'

In [41]:
hasil_hybrid_id

'sampah yang menumpuk di tempat pembuangan akhir sampah di kota bandung, jawa barat, dapat menyebabkan pencemaran lingkungan'

PENGUJIAN INFERENSI


In [42]:
import evaluate

In [43]:
input_text = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?


"""

In [44]:
import time

# Measure the time taken for TextRank summarization
num_runs = 10
total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    summary = f_textrank_en(input_text, 3)
    end_time = time.time()
    total_time += end_time - start_time

average_time = total_time / num_runs

print("Original Text:")
print(input_text)
print("\nTextRank Summary:")
print(summary)
print(f"\nAverage time for TextRank summarization: {average_time:.4f} seconds")


Original Text:


Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making

In [45]:
import time

# Measure the time taken for Pegasus summarization
num_runs = 10
total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    summary = f_pegasus_en(input_text)
    end_time = time.time()
    total_time += end_time - start_time

average_time = total_time / num_runs

print("Original Text:")
print(input_text)
print("\nPegasus Summary:")
print(summary)
print(f"\nAverage time for Pegasus summarization: {average_time:.4f} seconds")


Original Text:


Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making

In [46]:
import time

# Measure the time taken for Hybrid summarization
num_runs = 10
total_time = 0

for _ in range(num_runs):
    start_time = time.time()
    summary = f_hybrid_en(input_text, 2)
    end_time = time.time()
    total_time += end_time - start_time

average_time = total_time / num_runs

print("Original Text:")
print(input_text)
print("\nHybrid Summary:")
print(summary)
print(f"\nAverage time for Hybrid summarization: {average_time:.4f} seconds")


Original Text:


Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making

PENGUJIAN INFERENSI

In [47]:
# import and load dataset
from datasets import load_dataset

dataset_cnn = load_dataset("cnn_dailymail", '3.0.0')

print(f"Features in cnn : {dataset_cnn['train'].column_names}")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Features in cnn : ['article', 'highlights', 'id']


In [48]:
article_cnn = []
limit = 5
for i in range (limit):
  # Inisial reference
  article = dataset_cnn["test"][i]["article"]
  article_cnn.append(article)

In [49]:
# pengujian inferensi textrank
import time

# Measure the time taken for TextRank summarization for each article
n_time = []
total_time = 0

for article in article_cnn:
    start_time = time.time()
    summary = f_textrank_en(article, 5)
    end_time = time.time()
    total_time += end_time - start_time
    n_time.append(end_time - start_time)
average_time_per_article = total_time / len(article_cnn)

print("Original Articles:")
for idx, article in enumerate(article_cnn):
    print(f"\nArticle {idx + 1}:\n{article}")

print("\nTextRank Summaries:")
for idx, article in enumerate(article_cnn):
    summary = f_textrank_en(article, 5)
    print(f"\nSummary for Article {idx + 1}:\n{summary}")

print("\nTime for every article:")
for idx, time in enumerate(n_time):
    print(f"\nTime- {idx + 1}:\n{time:.4f}")

print(f"\nAverage time for TextRank summarization per article: {average_time_per_article:.4f} seconds")


Original Articles:

Article 1:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speak

In [50]:
# pengujian inferensi Pegasus
import time

# Measure the time taken for Pegasus summarization for each article
n_time = []
total_time = 0

for article in article_cnn:
    start_time = time.time()
    summary = f_pegasus_en(article)
    end_time = time.time()
    total_time += end_time - start_time
    n_time.append(end_time - start_time)
average_time_per_article = total_time / len(article_cnn)

print("Original Articles:")
for idx, article in enumerate(article_cnn):
    print(f"\nArticle {idx + 1}:\n{article}")

print("\nPegasus Summaries:")
for idx, article in enumerate(article_cnn):
    summary = f_pegasus_en(article)
    print(f"\nSummary for Article {idx + 1}:\n{summary}")

print("\nTime for every article:")
for idx, time in enumerate(n_time):
    print(f"\nTime- {idx + 1}:\n{time:.4f}")

print(f"\nAverage time for Pegasus summarization per article: {average_time_per_article:.4f} seconds")


Original Articles:

Article 1:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speak

In [51]:
# pengujian inferensi Hybrid
import time

# Measure the time taken for Hybrid summarization for each article
n_time = []
total_time = 0

for article in article_cnn:
    start_time = time.time()
    summary = f_hybrid_en(article, 2)
    end_time = time.time()
    total_time += end_time - start_time
    n_time.append(end_time - start_time)

average_time_per_article = total_time / len(article_cnn)

print("Original Articles:")
for idx, article in enumerate(article_cnn):
    print(f"\nArticle {idx + 1}:\n{article}")

print("\nHybrid Summaries:")
for idx, article in enumerate(article_cnn):
    summary = f_hybrid_en(article, 2)
    print(f"\nSummary for Article {idx + 1}:\n{summary}")

print("\nTime for every article:")
for idx, time in enumerate(n_time):
    print(f"\nTime- {idx + 1}:\n{time:.4f}")

print(f"\nAverage time for Hybrid summarization per article: {average_time_per_article:.4f} seconds")


Original Articles:

Article 1:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speak